In [1]:
import nltk 
import pandas as pd
from tqdm import tqdm
import stanza
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('punkt')

c:\Users\mixai\python projects\Visualisation\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mixai\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [3]:
df = pd.read_excel('fill_info.xlsx')
df_ml = df

full_corpus = df_ml["TEXT"].values

sentences = [sent for corp in full_corpus for sent in sent_tokenize(corp, language="russian")]

long_sents = [i for i in sentences if len(i) > 30]

nlp = stanza.Pipeline(lang='ru',
processors='tokenize,pos,lemma,ner,depparse')

2024-03-25 19:09:19 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-03-25 19:09:19 INFO: Downloaded file to C:\Users\mixai\stanza_resources\resources.json
2024-03-25 19:09:40 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |
| ner       | wikiner            |

2024-03-25 19:09:40 INFO: Using device: cpu
2024-03-25 19:09:40 INFO: Loading: tokenize
2024-03-25 19:09:41 INFO: Loading: pos
2024-03-25 19:09:41 INFO: Loading: lemma
2024-03-25 19:09:42 INFO: Loading: depparse
2024-03-25 19:09:42 INFO: Loading: ner
2024-03-25 19:09:43 INFO: Done loading processors!


In [4]:

triplets = []
for s in tqdm(long_sents):
    doc = nlp(s)
    for sent in doc.sentences:
        entities = [ent.text for ent in sent.ents]
        res_d = dict()
        temp_d = dict()
        for word in sent.words:
            temp_d[word.text] = {"head": sent.words[word.head-1].text, "dep": word.deprel, "id": word.id}
        for k in temp_d.keys():
            nmod_1 = ""
            nmod_2 = ""
            if (temp_d[k]["dep"] in ["nsubj", "nsubj:pass"]) & (k in entities):
                res_d[k] = {"head": temp_d[k]["head"]}
                
                for k_0 in temp_d.keys():
                    if (temp_d[k_0]["dep"] in ["obj", "obl"]) &\
                       (temp_d[k_0]["head"] == res_d[k]["head"]) &\
                        (temp_d[k_0]["id"] > temp_d[res_d[k]["head"]]["id"]):
                        res_d[k]["obj"] = k_0
                        break

                for k_1 in temp_d.keys():
                    if (temp_d[k_1]["head"] == res_d[k]["head"]) & (k_1 == "не"):
                        res_d[k]["head"] = "не "+res_d[k]["head"]
                
                if "obj" in res_d[k].keys():
                    for k_4 in temp_d.keys():
                        if (temp_d[k_4]["dep"] =="nmod") &\
                           (temp_d[k_4]["head"] == res_d[k]["obj"]):
                            nmod_1 = k_4
                            break
                            
                    for k_5 in temp_d.keys():
                        if (temp_d[k_5]["dep"] =="nummod") &\
                           (temp_d[k_5]["head"] == nmod_1):
                            nmod_2 = k_5
                            break
                    res_d[k]["obj"] = res_d[k]["obj"]+" "+nmod_2+" "+nmod_1

        if len(res_d) > 0:
            triplets.append([s, res_d])

100%|██████████| 526/526 [08:36<00:00,  1.02it/s]


In [6]:
nodes = set()
clear_triplets = []
for tr in triplets:
    for k in tr[1].keys():
        if "obj" in tr[1][k].keys():
            clear_triplets.append([tr[0], k, tr[1][k]['head'], tr[1][k]['obj']])
            nodes.add(k)
            nodes.add(tr[1][k]['obj'])


word_num = dict()
for c, word in enumerate(nodes):
    word_num[word] = c+1

import script_for_graph
import importlib
importlib.reload(script_for_graph)
from script_for_graph import header_text, tail_text

header_text += """\nvar nodes = new vis.DataSet([\n"""
for w in nodes:
    header_text += "{"
    header_text += f"""         id: {word_num[w]},
                                label: "{w}"\n"""
    header_text += "},"
header_text += "   ]);\n"

header_text += """var edges = new vis.DataSet(["""
for triplet in clear_triplets:
    info = '{"":"<>"}'.replace("<>",triplet[0])
    header_text += "{"
    header_text += f"""       from: {word_num[triplet[1]]}, 
                    to: {word_num[triplet[3]]}, 
                    arrows: "to",
                    label: "{triplet[2]}",
                    info: {info}\n"""
    header_text +="},"
header_text += "   ]);\n"

full_text = ""
full_text += header_text
full_text += tail_text

with open(f"page.html", "w", encoding="utf-8") as f: 
    f.write(full_text)